In [1]:
import numpy as np

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [3]:
def as_tuple(x):
    if not isinstance(x,tuple):
        return (x,)
    return x

In [4]:
class Variable:
    def __init__(self,data):
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError ('type:',type(data),'is not supported')
        self.data = data       # 変数の値の格納場所
        self.grad = None
        self.creator = None    # 変数の生成元の関数を保持しておく
    def set_creator(self,func):
        self.creator = func
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            print('grad is None. create',self.grad)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            gxs = as_tuple(gxs)
            print(f.name +".backward(",gys,") = ",gxs)
            if f.name == "Add":
                print(f.name + '\'s input is :',f.inputs[0].data ,"and",f.inputs[1].data)
            else:
                print(f.name + '\'s input is :',f.inputs[0].data)
            for x,gx in zip(f.inputs,gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                if x.creator is not None:        # (x.cretor is None == x が入力変数) なのでそこでストップする
                    funcs.append(x.creator)

In [5]:
class Function:
    def __call__(self,*inputs:Variable) -> Variable:
        xs = [x.data for x in inputs]
        ys = as_tuple(self.forward(*xs))
        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs    # 入力変数を保持しておく
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]   
    def forward(self,xs):
        raise NotImplementedError()
    def backward(self,gys):
        raise NotImplementedError()

In [6]:
class EXP(Function):
    name = "EXP"
    def forward(self,x):
        return np.exp(x)
    def backward(self,gy):
        return np.exp(self.inputs[0].data) * gy
    
    
def exp(x):
    return exp()(x)

In [7]:
class Square(Function):
    name = "Square"
    def forward(self,x):
        return x ** 2
    def backward(self,gy):
        return 2 * self.inputs[0].data * gy 

    
def square(x):
    return Square()(x)

In [8]:
class Add(Function):
    name = "Add"
    def forward(self,x0,x1):
        return x0 + x1
    def backward(self,gy):
        return gy,gy

def add(x0,x1):
    return Add()(x0,x1)

In [9]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
z = add(square(x),square(y))
z.backward()

grad is None. create 1.0
Add.backward( [array(1.)] ) =  (array(1.), array(1.))
Add's input is : 4.0 and 9.0
Square.backward( [array(1.)] ) =  (6.0,)
Square's input is : 3.0
Square.backward( [array(1.)] ) =  (4.0,)
Square's input is : 2.0


In [10]:
print(x.grad)
print(y.grad)
print(z.data)

4.0
6.0
13.0
